In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error, f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [ ]:
CSV_PATH = "../data/output/clean_data.csv"

# Load data
df = pd.read_csv(CSV_PATH)

y = df["cyberbullying"].values

# train/test split
split = int (0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

Index(['comment_text', 'cyberbullying'], dtype='object')


In [23]:
# Train Random Forest
model = RandomForestClassifier(n_estimators=50, random_state=42)
model.fit(X_train, y_train)

# Predictint on test data
y_pred = model.predict(X_test)

# Calculate metrics
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print metrics
print("Confusion Matrix:")
print(cm)
print(f"Accuracy: {accuracy:.4f}")
print(f"Mean Squared Error: {mse:.4f}")
print(f"F1 Score: {f1:.4f}")

Confusion Matrix:
[[28541   149]
 [ 1356  1869]]
Accuracy: 0.9528
Mean Squared Error: 0.0472
F1 Score: 0.7130
